In [2]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics

# Divide train into 90 % training and 10 % testing

In [2]:
all_features = ["text_ tokens", "tweet_id" ,"language", "tweet_timestamp", "engaged_with_user_id",\
               "engaging_user_id", "engagee_follows_engager", "retweet_with_comment_timestamp"]
train = pd.read_csv(open("s3://recsys-challenge-2020/training.tsv"), encoding="utf-8",
                    names = all_features, usecols= [0, 2, 7, 8, 9, 14, 19, 20], sep="\x01"
                   )
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100)))
val_set['retweet_with_comment_bool'] = val_set.retweet_with_comment_timestamp.fillna(0)
val_set[val_set.retweet_with_comment_bool != 0.0] = 1.0
gt = list(val_set.retweet_with_comment_bool)
pr = [0.5] * len(gt)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

In [ ]:
print(prauc)
print(rce)

In [3]:
all_features = ["tweet_timestamp", "retweet_with_comment_timestamp"]
train = pd.read_csv(open("s3://recsys-challenge-2020/training.tsv"), encoding="utf-8",
                    names = all_features, usecols= [8, 22], sep="\x01"
                   )
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()
val_set['retweet_with_comment_bool'] = val_set.retweet_with_comment_timestamp.fillna(0)
val_set.loc[val_set.retweet_with_comment_bool != 0.0, 'retweet_with_comment_bool'] = 1.0
gt = list(val_set.retweet_with_comment_bool)
pr_df = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f1/retweet_with_comment/val_set_result_sgd.csv',\
                      names = ['probability_retweet_with_comment'])
pr = list(pr_df.probability_retweet_with_comment)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

In [4]:
prauc

0.024704815192162485

In [5]:
rce

-4.816604111383782

In [3]:
all_features = ["tweet_timestamp", "retweet_with_comment_timestamp"]
train = pd.read_csv(open("s3://recsys-challenge-2020/training.tsv"), encoding="utf-8",
                    names = all_features, usecols= [8, 22], sep="\x01"
                   )
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()
val_set['retweet_with_comment_bool'] = val_set.retweet_with_comment_timestamp.fillna(0)
val_set.loc[val_set.retweet_with_comment_bool != 0.0, 'retweet_with_comment_bool'] = 1.0
gt = list(val_set.retweet_with_comment_bool)
pr_df = pd.read_csv\
('/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/retweet_with_comment/val_set.output',\
                      names = ['probability_retweet_with_comment'])
pr = list(pr_df.probability_retweet_with_comment)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

##### ./ffm-train -p ../retweet_with_comment/val_set.ffm -l 0.00002 --auto-stop -s 95 ../retweet_with_comment/train_set.ffm
##### Auto-stop. Use model at 3th iteration.

In [4]:
prauc

0.0329026409360537

In [5]:
rce

5.55414524871225

In [5]:
gt_df = pd.read_csv\
('/home/ubuntu/recsys_challenge_2020/classification_type_models/f1/retweet_with_comment/val_set_gt.csv',\
 names = ['retweet_with_comment_bool'])
gt = list(gt_df.retweet_with_comment_bool)
pr_df = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f1/retweet_with_comment/val_set_result_sgd.csv',\
                      names = ['probability_retweet_with_comment'])
pr = list(pr_df.probability_retweet_with_comment)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

In [6]:
prauc

0.023683919635709903

In [7]:
rce

-5.630098535096306

In [3]:
gt_df = pd.read_csv\
('/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/retweeet_with_comment/val_set_gt.csv',\
 names = ['retweet_with_comment_bool'])
gt = list(gt_df.retweet_with_comment_bool)
pr_df = pd.read_csv\
('/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/retweeet_with_comment/val_set.output',\
                      names = ['probability_retweet_with_comment'])
pr = list(pr_df.probability_retweet_with_comment)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

In [5]:
prauc

0.017824040495010313

In [6]:
rce

3.0459072186959424

In [4]:
prauc

0.010204908168066436

In [5]:
rce

-1475.7211245169601